# Degrees of Separation
Brief: BFS over people/movies graphs to find the shortest path between two actors.
Data: CSVs in small/ or large/ (people.csv, movies.csv, stars.csv).
Run: from degrees/, `python degrees.py` then enter two names; the program returns the chain of co-starring links.
Notes: picks the first matching name ID unless you disambiguate; use small/ for quicker tests.

## Frontier utilities
Helper `Node`, `StackFrontier`, and `QueueFrontier` classes used by BFS/DFS for storing states and reconstructing paths.

In [36]:
class Node() : 
    def __init__(self,state,parent,action) -> None:
        self.state=state
        self.parent=parent
        self.action=action
        
class StackFrontier() :
    def __init__(self) -> None:
        self.frontier = []
        
    def add(self, node) :
        self.frontier.append(node)
        
    def contains_state(self, state) :
        return any(state==node.state for node in self.frontier)
    
    def empty(self) :
        return len(self.frontier) == 0
    
    def remove(self) :
        if self.empty() :
            raise Exception("stack is empty")
        else :
            node = self.frontier[-1]
            self.frontier = self.frontier[ : -1]
            return node
        
class QueueFrontier(StackFrontier) :
    def remove(self) :
        if self.empty() :
            raise Exception("queue is empty")
        else :
            node = self.frontier[0]
            self.frontier = self.frontier[1:]
            return node
        

## Data loading and shortest-path search
Parses CSVs into in-memory graphs, resolves ambiguous names, and runs BFS (`shortest_path`) to find the lowest-degree connection between two actors.

In [37]:
import csv

# map for names with person_id
names = {}

#map for person_name as a key and their info {name, birth, set of movies_ids} as value
people = {}

#map for movie_id as a key and its info as a value {title, year, set of stars id's}
movies = {}

def load_data(directory) :
    
    #read people
    with open(f"{directory}/people.csv", encoding="utf-8") as f :
        reader = csv.DictReader(f)
        for row in reader :
            
            if row["name"].lower() not in names :
                names[row["name"].lower()] = {row["id"]}
            else:
                names[row["name"].lower()].add(row["id"])
                
            people[row["id"]] = {
                "name" : row["name"].lower(),
                "birth" : row["birth"],
                "movies" : set()
            }
    
    #read movies        
    with open(f"{directory}/movies.csv", encoding="utf-8") as f:
        reader = csv.DictReader(f)
        for row in reader :
            movies[row["id"]] = {
                "title" : row["title"].lower(),
                "year" : row["year"],
                "stars" : set()
            }
            
    #read stars
    with open(f"{directory}/stars.csv", encoding="utf-8") as f:
        reader = csv.DictReader(f)
        for row in reader :
            #try and excpet
            try :
                people[row["person_id"]]["movies"].add(row["movie_id"])
                movies[row["movie_id"]]["stars"].add(row["person_id"])
                
            except KeyError:
                print("there's an error")
                
                
def person_id_from_name(name) :
    person_ids = list(names.get(name.lower(), set()))
    
    if len(person_ids) <= 1 :
        return person_ids[0] if len(person_ids) else None
    else :
        print("here's a list of all actors with that name, which one your'e choosing?")
        for id in person_ids :
            print(f"ID : {id}, Name : {people[id]['name']}, Birth {people[id]['birth']}")
        try : 
            person_id = input("person id :")
            if person_id in person_ids :
                return person_id
        except ValueError : pass
        
        else : return None
        

def neighbours_for_person(id) :
    neighbours = set()
    for movie_id in people[id]["movies"] :      
        for star_id in movies[movie_id]["stars"] :
            neighbours.add((movie_id, star_id))
    return neighbours            
                

def shortest_path(src, dest) :
    
    start = Node(src, None, None)
    frontier = QueueFrontier()
    explored = set()
    frontier.add(start)
    
    while not frontier.empty() :
        
        current = frontier.remove()
        
        
        if current.state == dest :
            actions = []
            mutuals = []
            while current.parent is not None :
                actions.append(current.action)
                mutuals.append(current.state)
                current = current.parent
                
            actions.reverse()
            mutuals.reverse()

            return (mutuals, actions), len(mutuals)
        
        explored.add(current.state)
        for action, state in neighbours_for_person(current.state) :
            if state not in explored and not frontier.contains_state(state) :
                temp = Node(state, current, action)
                frontier.add(temp)
                
    return ([], []), 0

load_data("small")


source = input("name : ")
target = input("name : ")
source = person_id_from_name(source)
target = person_id_from_name(target)

if source and target :
    mutuals, degree = shortest_path(source, target)

    print(f"{degree} degrees of separation.")
    for index in range(len(mutuals[0])) :
        if index == 0 :
            print(f"{index+1} : {people[source]['name']} and {people[mutuals[0][index]]['name']} starred in {movies[mutuals[1][index]]['title']}")
        else :
            print(f"{index+1} : {people[mutuals[0][index-1]]['name']} and {people[mutuals[0][index]]['name']} starred in {movies[mutuals[1][index]]['title']}")
else :
    print("one of the two actors is not available in our data")


['112384', '104257', '95953'] ['102', '129', '596520'] 101
3 degrees of separation.
1 : tom hanks and kevin bacon starred in apollo 13
2 : kevin bacon and tom cruise starred in a few good men
3 : tom cruise and gerald r. molen starred in rain man
